# Crosstabs Example

In [6]:
df = pd.read_csv('test_data.csv')

In [7]:
df = df.sort_values(by='score', ascending=False)

In [26]:
# 10% cutoff
cutoff = 0.1

n_features = 1

ncols = 4

outfile_name = 'example.html'


In [9]:
# df_highrisk = df.iloc[0:cutoff,:]
# df_lowrisk = df.iloc[cutoff:, :]

In [10]:
# df_lowrisk.shape

(450, 92)

In [11]:
# df_highrisk.shape

(50, 92)

In [27]:
cutoff_idx = int(pd.np.floor(cutoff*len(df)))

# generate the histograms of the top-risk predictions,
# and of the remaining lower-risk predictions
myplots, myfeatures = plot_feature_histograms(
                [df.iloc[:cutoff_idx, :n_features],
                 df.iloc[cutoff_idx:, :n_features]],
                ['top_%d_predictions'%cutoff_idx, 
                 'bottom_%d_predictions'%(len(df)-cutoff_idx)],
                colors=["red", "green"])

/home/bene/DSaPP/env35/lib/python3.5/site-packages/bokeh/plotting/helpers.py:535: UserWarning: WheelZoomTool are being repeated
  warnings.warn("%s are being repeated" % ",".join(repeated_tools))


In [28]:
# finally, save the HTML       
save(gridplot(*myplots, ncols=ncols, plot_width=600, plot_height=400, 
    toolbar_location='above'),outfile_name, resources=CDN, 
    title='feature_densities')

'/home/bene/DSaPP/hitchhikers-guide/tech-tutorials/model_eval/5_bias/example.html'

In [41]:
df.score.rank(method='max')

0      500.0
1      499.0
2      498.0
3      497.0
4      496.0
5      495.0
6      494.0
7      494.0
9      494.0
8      494.0
10     490.0
11     490.0
12     490.0
13     490.0
14     486.0
15     486.0
17     484.0
18     484.0
16     484.0
20     481.0
21     481.0
22     481.0
19     481.0
27     477.0
23     477.0
24     477.0
25     477.0
26     477.0
28     477.0
29     477.0
       ...  
473     31.0
470     31.0
471     31.0
469     31.0
474     26.0
475     26.0
476     26.0
477     26.0
478     26.0
479     26.0
480     20.0
481     20.0
485     18.0
488     18.0
487     18.0
486     18.0
484     18.0
483     18.0
482     18.0
489     11.0
490     11.0
491     11.0
492     11.0
493     11.0
494      6.0
495      6.0
496      4.0
497      3.0
498      3.0
499      1.0
Name: score, dtype: float64

In [1]:
from plotter import *

In [2]:
df1, df2 = fetch_and_split_data('test_data.csv', cutoff=.1)

In [3]:
df1.shape

(52, 90)

In [4]:
df2.shape

(448, 90)

In [5]:
df = pd.read_csv('test_data.csv')

In [6]:
df.shape

(500, 92)

In [7]:
df1

,score,age,gender_female,incident_rate,random_feature_1,random_feature_2,random_feature_3,random_feature_4,random_feature_5,random_feature_6,...,random_feature_80,random_feature_81,random_feature_82,random_feature_83,random_feature_84,random_feature_85,random_feature_86,random_feature_87,random_feature_88,random_feature_89
0,0.610,45.707206,0,0.936189,0.0,0.0,0.0,1.0,0.0,0.0,...,1.935272,1.604414,-3.259050,-1.907094,-3.194122,5.667859,-1.803107,0.597377,0.131608,2.221849
1,0.600,69.079261,0,0.973462,1.0,0.0,0.0,1.0,0.0,1.0,...,1.800333,0.042844,-0.121097,-0.019635,-0.685809,-2.677701,2.040171,-0.126061,-0.249940,-3.261983
2,0.590,64.501325,0,0.876714,0.0,1.0,0.0,1.0,0.0,0.0,...,-1.455560,-0.889955,3.308929,-1.094386,3.820828,-1.493890,-3.059079,-1.554068,-0.264483,-1.483651
3,0.565,57.689446,0,0.890024,0.0,0.0,0.0,1.0,0.0,1.0,...,-0.770940,1.698292,-2.198866,0.312003,1.420848,-2.711358,-1.819957,-0.011389,-0.126452,-1.449900
4,0.560,30.877582,0,0.839481,0.0,1.0,0.0,1.0,0.0,0.0,...,-1.704050,1.438087,-0.301900,-1.079290,-2.121427,-6.733863,-2.015317,1.256779,0.106049,-3.229138
5,0.550,46.918857,0,0.821401,1.0,0.0,0.0,1.0,0.0,0.0,...,0.054140,0.223053,2.062282,-5.859959,5.135702,4.401903,-0.847833,-0.343842,0.400157,2.798165
6,0.545,65.139218,0,0.176374,0.0,1.0,0.0,1.0,0.0,0.0,...,0.388407,-0.201271,3.949461,-3.950919,1.108758,7.673945,1.958801,-0.390058,0.068962,1.105786
7,0.545,23.121496,0,0.835610,0.0,0.0,0.0,1.0,0.0,1.0,...,1.281093,-1.619812,0.308895,0.046728,-4.741117,0.472311,0.870932,-0.997469,-0.015326,5.003751
9,0.545,40.755420,0,0.871950,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.663325,-2.318051,1.448330,0.961425,-0.533420,-0.751356,4.055057,-0.304116,-0.330331,-6.239099
8,0.545,80.173084,0,0.352738,0.0,0.0,0.0,1.0,0.0,0.0,...,2.137119,-1.449521,-0.736680,-1.384991,6.199024,-4.105965,2.830056,0.019276,-0.322225,-8.655330


In [17]:
list(map(str,df.columns))

['entity_id',
 'true_label',
 'score',
 'age',
 'gender_female',
 'incident_rate',
 'random_feature_1',
 'random_feature_2',
 'random_feature_3',
 'random_feature_4',
 'random_feature_5',
 'random_feature_6',
 'random_feature_7',
 'random_feature_8',
 'random_feature_9',
 'random_feature_10',
 'random_feature_11',
 'random_feature_12',
 'random_feature_13',
 'random_feature_14',
 'random_feature_16',
 'random_feature_17',
 'random_feature_18',
 'random_feature_19',
 'random_feature_20',
 'random_feature_21',
 'random_feature_22',
 'random_feature_23',
 'random_feature_24',
 'random_feature_25',
 'random_feature_26',
 'random_feature_27',
 'random_feature_28',
 'random_feature_29',
 'random_feature_31',
 'random_feature_32',
 'random_feature_33',
 'random_feature_34',
 'random_feature_35',
 'random_feature_36',
 'random_feature_37',
 'random_feature_38',
 'random_feature_39',
 'random_feature_40',
 'random_feature_41',
 'random_feature_42',
 'random_feature_43',
 'random_feature_44',
 '